In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import scipy.stats as ss
from scipy.interpolate import interp1d
from scipy.special import expit
from fractions import Fraction
from empiricaldist import Pmf, Cdf
import matplotlib.pyplot as plt
from collections import Counter
import statsmodels.formula.api as smfa

In [5]:
df = pd.read_csv(Path.cwd() / 'data' / '2239075.csv', usecols= ['DATE', 'SNOW'], parse_dates=['DATE'])
df.head()

,DATE,SNOW
0,1967-05-11,0.0
1,1967-05-12,0.0
2,1967-05-13,0.0
3,1967-05-14,0.0
4,1967-05-15,0.0


In [6]:
df['YEAR'] = df['DATE'].dt.year

In [7]:
snow = df.groupby('YEAR')['SNOW'].sum()
snow.head()

YEAR
1967    28.6
1968    44.7
1969    99.2
1970    66.8
1971    54.6
Name: SNOW, dtype: float64

In [9]:
pmf_snowfall = Pmf.from_seq(snow)
pmf_snowfall.head()

,probs
12.2,0.018519
12.5,0.018519
24.1,0.018519


In [10]:
mean, std = pmf_snowfall.mean(), pmf_snowfall.std()

In [11]:
dist = ss.norm(mean, std)
qs = pmf_snowfall.qs
ps = dist.cdf(qs)
ps

array([0.03171465, 0.03251042, 0.07814883, 0.09940183, 0.10532122,
       0.120115  , 0.12309132, 0.14962096, 0.18557042, 0.18754804,
       0.19254879, 0.209091  , 0.24342151, 0.25511263, 0.27073895,
       0.27687382, 0.31766867, 0.32426615, 0.34574301, 0.35530276,
       0.36911765, 0.37889092, 0.38451108, 0.40865501, 0.41295481,
       0.42736024, 0.42880646, 0.44914526, 0.45644451, 0.48429498,
       0.49017261, 0.52103551, 0.52250349, 0.54301419, 0.56195893,
       0.57932221, 0.58938222, 0.70385178, 0.74088187, 0.74681241,
       0.76643005, 0.80403479, 0.83222423, 0.83771327, 0.84746935,
       0.85091848, 0.91088915, 0.91148171, 0.91947193, 0.96194621,
       0.96574509, 0.98843349, 0.99809832])

In [12]:
data = snow.reset_index()

In [16]:
offset = round(data['YEAR'].mean(), 0)
data['x'] = data['YEAR'] - offset
data['y'] = data['SNOW']

In [17]:
formula = 'y ~ x'
results = smfa.ols(formula, data=data).fit()
results.params

Intercept    62.780489
x             0.423941
dtype: float64

In [18]:
results.resid.std()

26.56611464774299

In [19]:
def make_uniform(qs, name=None, **options):
    pmf = Pmf(1.0, qs, **options)
    pmf.normalize()
    if name:
        pmf.index.name = name
    return pmf

In [23]:
qs1 = np.linspace(-0.5, 1.5, 51)
prior_slope = make_uniform(qs1, name='Slope')
qs2 = np.linspace(54, 75, 40)
prior_inter = make_uniform(qs2, name='Intercept')
qs3 = np.linspace(20, 35, 31)
prior_sigma = make_uniform(qs3, name='Sigma')

In [21]:
def make_joint(pmf1, pmf2):
    X, Y = np.meshgrid(pmf1, pmf2)
    return pd.DataFrame(X * Y, columns=pmf1.qs, index=pmf2.qs)

In [35]:
def make_joint3(pmf1, pmf2, pmf3):
    joint2 = make_joint(pmf2, pmf1)
    joint2_pmf = Pmf(joint2.stack())
    joint3 = make_joint(pmf3, joint2_pmf)
    joint3.index = pd.MultiIndex.from_tuples(joint3.index)
    return Pmf(joint3.stack())

In [36]:
prior = make_joint3(prior_slope, prior_inter, prior_sigma)
prior.head()

-0.5  54.0  20.0    0.000016
            20.5    0.000016
            21.0    0.000016
Name: , dtype: float64

In [37]:
inter = 64
slope = 0.51
sigma = 25

In [38]:
xs = data['x']
ys = data['y']

In [39]:
expected = inter + slope * xs
resid = ys - expected

In [41]:
densities = ss.norm(0, sigma).pdf(resid)
likelihood = densities.prod()

In [42]:
likelihood = prior.copy()

In [43]:
for slope, inter, sigma in prior.index:
    expected = inter + slope * xs
    resid = ys - expected
    #densities = ss.norm.pdf(resid, 0, sigma)
    densities = ss.norm(0, sigma).pdf(resid)
    likelihood[slope, inter, sigma] = densities.prod()

In [44]:
posterior = prior * likelihood
posterior.normalize()

5.446733376615371e-112

In [45]:
posterior_slope = posterior.marginal(0)
posterior_inter = posterior.marginal(1)
posterior_sigma = posterior.marginal(2)

In [51]:
url = 'https://en.wikipedia.org/wiki/Marathon_world_record_progression#Men'
table = pd.read_html(url)[0]

In [56]:
table['date'] = pd.to_datetime(table['Date'], errors='coerce')
table['time'] = pd.to_timedelta(table['Time'])
table['y'] = 26.2 / table['time'].dt.total_seconds() * 3600

In [57]:
def plot_speeds(df):
    plt.axhline(13.1, color='C5', ls='--')
    plt.plot(df['date'], df['y'], 'o', 
             label='World record speed', 
             color='C1', alpha=0.5)

In [58]:
recent = table['date'] > pd.to_datetime('1970')
data = table.loc[recent].copy()

In [59]:
offset = pd.to_datetime('1995')
timedelta = table['date'] - offset
data['x'] = timedelta.dt.total_seconds() / 3600 / 24 / 365.24

In [61]:
formula = 'y ~ x'
results = smfa.ols(formula, data=data).fit()
results.params

Intercept    12.464040
x             0.015931
dtype: float64

In [62]:
results.resid.std()

0.04419653543387639

In [63]:
qs1 = np.linspace(0.012, 0.018, 51)
prior_slope = make_uniform(qs1, name='Slope')
qs2 = np.linspace(12.4, 12.5, 41)
prior_inter = make_uniform(qs2, name='Intercept')
qs3 = np.linspace(0.01, 0.21, 31)
prior_sigma = make_uniform(qs3, name='Sigma')

In [64]:
prior = make_joint3(prior_slope, prior_inter, prior_sigma)
prior.head()

0.012  12.4  0.010000    0.000015
             0.016667    0.000015
             0.023333    0.000015
Name: , dtype: float64

In [65]:
xs = data['x']
ys = data['y']

In [66]:
likelihood = prior.copy()

In [67]:
for slope, inter, sigma in prior.index:
    expected = inter + slope * xs
    resid = ys - expected
    #densities = ss.norm.pdf(resid, 0, sigma)
    densities = ss.norm(0, sigma).pdf(resid)
    likelihood[slope, inter, sigma] = densities.prod()

In [68]:
posterior = prior * likelihood
posterior.normalize()

1161389020603.8816

In [69]:
posterior_slope = posterior.marginal(0)
posterior_inter = posterior.marginal(1)
posterior_sigma = posterior.marginal(2)

In [70]:
sample = posterior.choice(101)

In [71]:
xs = np.arange(-25, 50)
pred = np.empty((len(sample), len(xs)))

In [72]:
for i, (slope, inter, sigma) in enumerate(sample):
    epsilon = ss.norm(0, sigma).rvs(len(xs))
    pred[i] = inter + slope * xs + epsilon

In [74]:
low, median, high = np.percentile(pred, [5, 50, 95], axis=0)